# Imports #

In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')
# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)

# Data #

In [ ]:
fuel = pd.read_csv('../input/dl-course-data/fuel.csv')

In [ ]:
fuel.head()
fuel.info()
fuel.describe()

In [ ]:
sns.pairplot(fuel);

# Preprocessing #

In [ ]:
X = fuel.copy()
y = X.pop('FE')

preprocessor = make_column_transformer(
    (StandardScaler(),
     make_column_selector(dtype_include=np.number)),
    (OneHotEncoder(sparse=False),
     make_column_selector(dtype_include=object)),
)

X_train, X_valid, y_train, y_valid = \
    train_test_split(X, y, train_size=0.75)

X_train = preprocessor.fit_transform(X_train)
X_valid = preprocessor.transform(X_valid)
y_train = np.log(y_train) # log transform target instead of standardizing
y_valid = np.log(y_valid)

In [ ]:
ds_train_ = tf.data.Dataset.from_tensor_slices((X_train, y_train))
ds_valid_ = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))

BATCH_SIZE = 128
NUM_FEATURES = X_train.shape[1]
AUTO = tf.data.experimental.AUTOTUNE
ds_train = (ds_train_
            .cache()
            .shuffle(10000)
            .batch(BATCH_SIZE)
            .prefetch(AUTO))

ds_valid = (ds_valid_
            .batch(BATCH_SIZE)
            .cache()
            .prefetch(AUTO))

# Linear Model #

In [ ]:
model = keras.Sequential([
    layers.InputLayer(input_shape=(NUM_FEATURES, )),
    layers.Dense(1),
])
model.compile(
    optimizer='sgd',
    loss='mae',
    metrics=['mae'],
)

early_stopping = keras.callbacks.EarlyStopping(patience=10, min_delta=0.0001)
EPOCHS = 1000
history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    verbose=0,
)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[2:, ['loss', 'val_loss']].plot();

In [ ]:
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()));

# Deep Model #

In [ ]:
model = keras.Sequential([
    layers.InputLayer(input_shape=(NUM_FEATURES, )),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),    
    layers.Dense(64, activation='relu'),
    layers.Dense(1),
])
model.compile(
    optimizer='adam',
    loss='mae',
    metrics=['mae', 'mse'],
)

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(patience=20, min_delta=1e-4)
EPOCHS = 1000
history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    verbose=0,
)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[0:, ['loss', 'val_loss']].plot();

In [ ]:
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()))

# Dropout #

In [ ]:
model = keras.Sequential([
    layers.InputLayer(input_shape=(NUM_FEATURES, )),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(1),
])
model.compile(
    optimizer='adam',
    loss='mae',
    metrics=['mae', 'mse'],
)

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(patience=20, min_delta=1e-4)
EPOCHS = 1000
history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    verbose=0,
)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[0:, ['loss', 'val_loss']].plot();

In [ ]:
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()))

# Batch Normalization #

In [ ]:
model = keras.Sequential([
    layers.InputLayer(input_shape=(NUM_FEATURES, )),
    layers.Dense(128),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.1),
    layers.Dense(128),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.1),
    layers.Dense(128),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.1),
    layers.Dense(1),
])
model.compile(
    optimizer='adam',
    loss='mae',
    metrics=['mae', 'mse'],
)

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(patience=50, min_delta=1e-4)
EPOCHS = 1000
history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    verbose=0,
)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[0:, ['loss', 'val_loss']].plot();

In [ ]:
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()))

# Swish Activation #

In [ ]:
model = keras.Sequential([
    layers.InputLayer(input_shape=(NUM_FEATURES, )),
    layers.Dense(64, activation='swish'),
    layers.Dense(64, activation='swish'),    
    layers.Dense(64, activation='swish'),
    layers.Dense(1),
])
model.compile(
    optimizer='adam',
    loss='mae',
    metrics=['mae', 'mse'],
)

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(patience=50, min_delta=1e-4)
EPOCHS = 1000
history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    verbose=0,
)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[5:, ['loss', 'val_loss']].plot();

In [ ]:
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()))